### Run the following chunk of code to import any libraries or packages necessary to run the rest of the code.

In [ ]:
import numpy as np
import os as os
import re as re
import pandas as pd

### Run the following chunk of code to calculate the sum of the time averaged (TA) forces (Electrostatic & Hydrophobic).

The following code will calculate the sum of the electrostatic and hydrophobic forces. Note, the user will need to adjust the directory variables to match their directory structure, or adjust their directory structure to match that of the code.

In [ ]:
base_Directory = ""

phos = 4
chain_Length = 150

input_Working_Directory_One = base_Directory + "Inputs/Electrostatic/" + str(phos) + "x_Phos/"
input_Working_Directory_Two = base_Directory + "Inputs/Hydrophobic/" + str(phos) + "x_Phos/"
output_Working_Directory_One = base_Directory + "Outputs/Electrostatic/Mean/" + str(phos) + "x_Phos/"
output_Working_Directory_Two = base_Directory + "Outputs/Hydrophobic/Mean/" + str(phos) + "x_Phos/"
output_Working_Directory_Three = base_Directory + "Outputs/Electrostatic/SD/" + str(phos) + "x_Phos/"
output_Working_Directory_Four = base_Directory + "Outputs/Hydrophobic/SD/" + str(phos) + "x_Phos/"

force_Conversion = 1.661*10**(-12) #N Per kJ/(nm*mol)

concentrations = [10,11,12,13,14,15,16,17,18,19,20]
protein_Counts = [125,140,150,160,180,200,210,220,240,250,270]

for file in os.listdir(input_Working_Directory_One):
    print(file)
    electrostatic_AA_Forces = np.load(input_Working_Directory_One + file)
    hydrophobic_AA_Forces = np.load(input_Working_Directory_Two + file.replace("Electrostatic_Forces", "Hydrophobic_Forces"))

    concentration = int(re.split("5x_Protein_|_uM",file)[1])
    protein_Count = [protein_Counts[i] for i in range(len(concentrations)) if concentrations[i] == concentration][0]
    AA_Count = protein_Count*chain_Length
    
    hydrophobic_Sum_Total = 0
    electrostatic_Sum_Total = 0
    hydrophobic_Sum_Error = 0
    electrostatic_Sum_Error = 0
    
    rows = np.shape(electrostatic_AA_Forces)[0]
    columns = np.shape(electrostatic_AA_Forces)[1]
    for i in range(rows):
        if i%100 == 0:
            print(i)
        index_One_Protein = int(np.floor(i/chain_Length))
        for j in range(columns):
            index_Two_Protein = int(np.floor(j/chain_Length))
            if index_One_Protein != index_Two_Protein:
                if hydrophobic_AA_Forces[i,j] < 0:
                    hydrophobic_Sum_Total += hydrophobic_AA_Forces[i,j]*force_Conversion
                electrostatic_Sum_Total += electrostatic_AA_Forces[i,j]*force_Conversion

    
    np.save(output_Working_Directory_One + file.replace("Electrostatic_Forces","Force_Sums_Electrostatic"), electrostatic_Sum_Total)
    np.save(output_Working_Directory_Two + file.replace("Electrostatic_Forces","Force_Sums_Hydrophobic"), hydrophobic_Sum_Total)